In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# def dfrel_to_ho_set():
#     return


In [3]:

# from logging.config import _RootLoggerConfiguration

import os
from pathlib import Path

def create_folder_if_not_exists(dr):
    dr = Path(dr)
    constructed_dir = str(Path('/').resolve())
    made = False
    
#     print('constructed_dir', constructed_dir)
    for fold in dr.parts:
        if len(fold) == 0:
            continue
        constructed_dir = str((Path(constructed_dir) / fold).resolve())
#         print('constructed_dir', constructed_dir)
        if os.path.isdir(constructed_dir):
            # print(f'Directory already exists:', constructed_dir)
            pass
        else:
            # print(f'Creating directory:', constructed_dir)
            os.mkdir(constructed_dir)
            made = True
    if made:
        print(f'Created directory:', constructed_dir)
    return

# create_folder_if_not_exists((Path.home() / 'Desktop/nada/folder2').resolve())

In [4]:
def create_all_folders(base_folder, prefix, best_coeffs_folder,
                       best_coeffs_npy_folder, best_reconstruct_folder, best_resids_npy_folder, 
                       best_combined_coeffs_folder, all_models_folder, 
                       all_coeffs_folder, all_reconstruct_folder):
    create_folder_if_not_exists(str(Path(base_folder/prefix).resolve()))
    create_folder_if_not_exists(str(Path(base_folder/prefix/best_coeffs_folder).resolve()))
    create_folder_if_not_exists(str(Path(base_folder/prefix/best_coeffs_npy_folder).resolve()))
    create_folder_if_not_exists(str(Path(base_folder/prefix/best_resids_npy_folder).resolve()))
    create_folder_if_not_exists(str(Path(base_folder/prefix/best_combined_coeffs_folder).resolve()))
    create_folder_if_not_exists(str(Path(base_folder/prefix/all_models_folder/'coeffs').resolve()))
    create_folder_if_not_exists(str(Path(base_folder/prefix/all_models_folder/'intercepts').resolve()))
    create_folder_if_not_exists(str(Path(base_folder/prefix/all_coeffs_folder).resolve()))
    create_folder_if_not_exists(str(Path(base_folder/prefix/all_reconstruct_folder).resolve()))
    create_folder_if_not_exists(str(Path(base_folder/prefix/best_reconstruct_folder).resolve()))

In [5]:
import sys

# From stack overflow
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

In [6]:
def func_for_cprofile():
    from sglm.models import sglm_cv
    import itertools
    import time
    import glob
    import numpy as np
    import pandas as pd
    import numpy as np
    from sglm.features import gen_signal_df as gsd
    from sglm.features import build_features as bf
    from sglm.features import setup_model_fit as smf
    from sglm.models import sglm_cv
    from sglm import models
    from sglm.visualization import visualize
    from sglm.models import train_model
    from sglm.models import eval
    from sglm import features

    neg_order = -20
    pos_order = 20

    fix_training = True
    # fix_training = False
    ft_str = '-ft' if fix_training else ''

    # multifile_fit = 'single' #False
    # multifile_fit = 'all' #True
    # multifile_fit = 'by_mouse'
    # multifile_fit_list = ['all']
    # multifile_fit_list = ['by_mouse']
    multifile_fit_list = ['all']
    # multifile_fit_list = ['by_mouse', 'all']
    # multifile_fit_list = ['all']
    # multifile_fit_list = ['single']

    # base_prefix = 'trial_abm_slOff'
    # base_prefix = 'bidir_pred_bidir'
    # base_prefix = 'dh'
    # base_prefix = 'gr-dual-n62'
    # base_prefix = 'bidir-ctrl-nslO'
    # base_prefix = 'DA-DA-pred'
    # base_prefix = 'ACH-to-DA'
    # base_prefix = 'DA-to-ACH__'

    # base_prefix = 'ind'
    # base_prefix = 'resid'
    # base_prefix = 'resid-chk'
    # base_prefix = 'resid-chk5'
    # base_prefix = 'test-res_trk'


    # base_prefix = 'f1-bkwd-sel'
    # base_prefix = 'f1-bkwd-sel-gACH2-rl'
    # base_prefix = 'f1-bkwd-sel-rDA2'
    # base_prefix = 'f1-bkwd-sel-gDA2'

    # base_prefix = 'f1-bkwd-sel-hypPrm-rDA'
    # base_prefix = 'f1-bkwd-sel-hypPrm-gACH'

    tmp_y_col_setup = 'gACH'
    # base_prefix = f'f1-bkwd-sel-{tmp_y_col_setup}3'
    # base_prefix = 'f1-bkwd-sel-rDA3'
    # base_prefix = 'f1-bkwd-sel-gDA3'

    # base_prefix = f'mse-tr-te-invstg3-{tmp_y_col_setup}'
    # base_prefix = f'mse-tr-te-invstg-ho-{tmp_y_col_setup}'
    # base_prefix = f'mse-tr-te-invstg-all-{tmp_y_col_setup}'
    # base_prefix = f'mse-tr-te-lim-bnds-{tmp_y_col_setup}'

    # base_prefix = f'bs-words-{tmp_y_col_setup}'
    # base_prefix = f'only-all-10x-h'
    # base_prefix = f'oall-nTfn4'
    base_prefix = f'oall-cprofile17-hdf'


    # data_folder = 'test'
    # data_folder = 'fig5/g5'

    data_folder = 'fig1'
    # data_folder = 'fig3'
    # data_folder = 'fig3-dualhem'
    # data_folder = 'fig5/g1'
    # data_folder = 'fig5/g2'
    # data_folder = 'fig5/g3'
    # data_folder = 'fig5/g4'
    # data_folder = 'fig5/g5'

    # data_folder = 'fig3-dualhem'



    if data_folder == 'test':
        wt_used = [
            'WT63', 'WT64', 'WT53', 'WT69'
        ]

    elif data_folder == 'fig1':

        wt_used = [
                   'WT63', 'WT64', 'WT65', 'WT66', 'WT67', 'WT68', 'WT69', # DA
                   'WT57', 'WT58', 'WT59', 'WT60', 'WT61', 'WT53', 'WT55', 'WT56' # ACH
                   ]

    elif data_folder == 'fig3':
        wt_used = ['WT61', 'WT63', 'WT64', 'WT44', 'WT51']
    elif data_folder == 'fig3-dualhem':
        wt_used = ['WT63', 'WT64', 'WT65']

    elif data_folder == 'fig4/g1':
        wt_used = ['S1233', 'S1234', 'S1260', 'S1246', 'S1248']
    elif data_folder == 'fig4/g2':
        wt_used = ['S1194', 'S1195', 'S1214', 'S1258', 'S1259']

    elif data_folder == 'fig5/g1': # Drd2f/f control: S1417, 1419, 1421
        # wt_used = ['S1417', 'S1419', 'S1421']
        wt_used = ['S1417', 'S1419', 'S1421', 'S1460', 'S1462', 'S1473', 'S1474']
    elif data_folder == 'fig5/g2': # Chat Cre X Drd2f/f : S1416, 1418, 1420, 1422
        # wt_used = ['S1416', 'S1418', 'S1420', 'S1422']
        wt_used = ['S1416', 'S1418', 'S1420', 'S1459', 'S1461', 'S1470', 'S1471', 'S1472']
    elif data_folder == 'fig5/g3': # Chat Cre control: S1355-1358, S1374, S1376
        # wt_used = ['S1355', 'S1356', 'S1357', 'S1358', 'S1374', 'S1376']
        wt_used = ['S1355', 'S1356', 'S1357', 'S1358', 'S1374', 'S1376',
                   'S1448', 'S1449', 'S1450', 'S1451']
    elif data_folder == 'fig5/g4': # Chat Cre control: S1399-1401
        # wt_used = ['S1399', 'S1400', 'S1401']
        wt_used = ['S1399', 'S1400', 'S1401']
    elif data_folder == 'fig5/g5':
        # wt_used = ['S1355', 'S1356', 'S1357', 'S1358', 'S1374', 'S1376', 'S1399', 'S1400', 'S1401']
        wt_used = ['S1355', 'S1356', 'S1357', 'S1358', 'S1374', 'S1376',
                   'S1448', 'S1449', 'S1450', 'S1451'
                   'S1399', 'S1400', 'S1401']
    elif data_folder == 'fig5/g6':
        wt_used = []
    else:
        raise ValueError('Unimplemented figure values.')

    data_folder_src = data_folder if data_folder != 'test' else 'fig1'
    data_folder_join = '_'.join(data_folder.split('/'))


    # ### Backwards Selection
    # X_y_pairings = [
    #     {'X_cols': {
    #                 'rDA': (0, 0),
    #                },
    #      'y_col': tmp_y_col_setup},
    #     {'X_cols': {
    #                 'photometryCenterInIndex':(0,0),
    #                 'photometryCenterOutIndex':(0,0),
    #                 'photometrySideInIndex':(0,0),
    #                 'photometrySideInIndexr':(0,0),                
    #                 'photometrySideOutIndex':(0,0),
    #                 'sl': (0,0),
    #                 'spnnrOff': (0,0),
    #                },
    #      'y_col': tmp_y_col_setup + '_resid'},
    #     {'X_cols': {
    #                 'photometryCenterInIndex':(0,0),
    #                 'photometryCenterOutIndex':(0,0),
    #                 'photometrySideInIndex':(0,0),
    #                 'photometrySideInIndexr':(0,0),                
    #                 'photometrySideOutIndex':(0,0),
    #                 'sl': (0,0),
    #                 'spnnrOff': (0,0),
    #                 'rDA': (0, 0),
    #                },
    #      'y_col': tmp_y_col_setup},
    #     {'X_cols': {
    #                 'photometryCenterInIndex':(0,0),
    #                 'photometryCenterOutIndex':(0,0),

    #                 'photometrySideInIndexAA':(0,0), 'photometrySideInIndexAa':(0,0),
    #                 'photometrySideInIndexaA':(0,0), 'photometrySideInIndexaa':(0,0),
    #                 'photometrySideInIndexAB':(0,0), 'photometrySideInIndexAb':(0,0),
    #                 'photometrySideInIndexaB':(0,0), 'photometrySideInIndexab':(0,0),

    #                 'photometrySideOutIndex':(0,0),
    #                 'sl': (0,0),
    #                 'spnnrOff': (0,0),
    #                },
    #      'y_col': tmp_y_col_setup},
    # ]


    # ## Start by saying this model works
    # {
    #     'photometryCenterInIndex':(0,0),
    #     'photometryCenterOutIndex':(0,0),
    #     'photometrySideInIndex':(0,0),
    #     'photometrySideInIndexr':(0,0),                
    #     'photometrySideOutIndex':(0,0),
    #     'sl': (0,0),
    #     'spnnrOff': (0,0),
    # }
    # # Go through & drop one by one


    num_runs = 10

    ### Backwards Selection
    X_y_pairings_lst = []

    X_y_pairings_lst += [[
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),                
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gACH',
         'name': 'base-simple'
         },
        {'X_cols': {
                    # 'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gACH',
         'name': 'no-CI'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    # 'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gACH',
         'name': 'no-CO'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    # 'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gACH',
         'name': 'no-SI'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    # 'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gACH',
         'name': 'no-SIr'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    # 'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gACH',
         'name': 'no-SO'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    # 'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gACH',
         'name': 'no-sl'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    # 'spnnrOff': (0,0),
                   },
         'y_col': 'gACH',
         'name': 'no-spnnrOff'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndexAA':(0,0),
                    'photometrySideInIndexAa':(0,0),
                    'photometrySideInIndexaA':(0,0),
                    'photometrySideInIndexaa':(0,0),
                    'photometrySideInIndexAB':(0,0),
                    'photometrySideInIndexAb':(0,0),
                    'photometrySideInIndexaB':(0,0),
                    'photometrySideInIndexab':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gACH',
         'name': 'base-words'},

    ]]

    X_y_pairings_lst += [[

        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),                
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'rDA',
         'name': 'base-simple'
         },
        {'X_cols': {
                    # 'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'rDA',
         'name': 'no-CI'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    # 'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'rDA',
         'name': 'no-CO'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    # 'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'rDA',
         'name': 'no-SI'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    # 'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'rDA',
         'name': 'no-SIr'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    # 'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'rDA',
         'name': 'no-SO'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    # 'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'rDA',
         'name': 'no-sl'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    # 'spnnrOff': (0,0),
                   },
         'y_col': 'rDA',
         'name': 'no-spnnrOff'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndexAA':(0,0),
                    'photometrySideInIndexAa':(0,0),
                    'photometrySideInIndexaA':(0,0),
                    'photometrySideInIndexaa':(0,0),
                    'photometrySideInIndexAB':(0,0),
                    'photometrySideInIndexAb':(0,0),
                    'photometrySideInIndexaB':(0,0),
                    'photometrySideInIndexab':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'rDA',
         'name': 'base-words'},

    ]]

    X_y_pairings_lst += [[

        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),                
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gDA',
         'name': 'base-simple'
         },
        {'X_cols': {
                    # 'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gDA',
         'name': 'no-CI'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    # 'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gDA',
         'name': 'no-CO'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    # 'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gDA',
         'name': 'no-SI'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    # 'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gDA',
         'name': 'no-SIr'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    # 'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gDA',
         'name': 'no-SO'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    # 'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gDA',
         'name': 'no-sl'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndex':(0,0),
                    'photometrySideInIndexr':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    # 'spnnrOff': (0,0),
                   },
         'y_col': 'gDA',
         'name': 'no-spnnrOff'},
        {'X_cols': {
                    'photometryCenterInIndex':(0,0),
                    'photometryCenterOutIndex':(0,0),
                    'photometrySideInIndexAA':(0,0),
                    'photometrySideInIndexAa':(0,0),
                    'photometrySideInIndexaA':(0,0),
                    'photometrySideInIndexaa':(0,0),
                    'photometrySideInIndexAB':(0,0),
                    'photometrySideInIndexAb':(0,0),
                    'photometrySideInIndexaB':(0,0),
                    'photometrySideInIndexab':(0,0),
                    'photometrySideOutIndex':(0,0),
                    'sl': (0,0),
                    'spnnrOff': (0,0),
                   },
         'y_col': 'gDA',
         'name': 'base-words'},

    ]]


    plot_width = 2
    max_cols_len_lst = [max([len(_['X_cols']) for _ in inner_list]) for inner_list in X_y_pairings_lst]
    plot_rows_lst = [_//plot_width + (_%plot_width > 0)*1 for _ in max_cols_len_lst]

    drop_cols_basis = [
        'nTrial', 'nTrial_filenum',
        'cpn', 'cpx',
        'spnnr',
        'spxnr',
        'spnr', 'spxr',

        'photometryCenterInIndex', 'photometryCenterOutIndex',
        'photometrySideInIndexr', 'photometrySideInIndexnr',
        'photometrySideOutIndex', 'spnnrOff',

        'photometrySideInIndexAA', 'photometrySideInIndexAa',
        'photometrySideInIndexaA', 'photometrySideInIndexaa',
        'photometrySideInIndexAB', 'photometrySideInIndexAb',
        'photometrySideInIndexaB', 'photometrySideInIndexab',

        'sl',
        'slOff'
    ]

    dfrr_cols = ['nTrial', 'nTrial_filenum', 'nEndTrial', 'wi_trial_keep', 'gDA', 'gACH', 'rDA',
        'diffTrialNums', 'dupe',
        'photometryCenterInIndex', 'photometryCenterOutIndex',
        'photometrySideInIndexr', 'photometrySideInIndexnr',
        'photometrySideOutIndex', 'spnnrOff', 'sl',

        'photometrySideInIndexAA', 'photometrySideInIndexAa',
        'photometrySideInIndexaA','photometrySideInIndexaa',
        'photometrySideInIndexAB', 'photometrySideInIndexAb',
        'photometrySideInIndexaB','photometrySideInIndexab',
    ]

    # create_folder_if_not_exists(dir)

    from pathlib import Path

    # base_folder = '/home/josh/github-repos/sabatinilab-glm/sglm/outputs'
    base_folder = (Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs').resolve())
#     base_folder = '/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/sglm/outputs'
    base_folder = (Path(base_folder).resolve())

    # pr = path_resolve, ps = path_string
    pr = lambda x: str(Path(x).resolve())
    ps = lambda x: str(Path(x))

    ssave_folder = ps('models/ssave')
    all_models_folder = ps('models/all_models')
    all_data_folder = ps('models/all_data')
    all_reconstruct_folder = ps('reports/figures/all_reconstruct')
    all_coeffs_folder = ps('reports/figures/all_coeffs')
    best_reconstruct_folder = ps('reports/figures/best_reconstruct')
    best_coeffs_npy_folder = ps('reports/coeffs/best_coeffs')
    best_resids_npy_folder = ps('reports/residuals/best_resids')
    best_combined_coeffs_folder = ps('reports/coeffs/best_combined_coeffs')
    best_coeffs_folder = ps('reports/figures/best_coeffs')

    all_models_folder = 'models'

    avg_reconstruct_basename = 'arr'
    all_betas_basename = 'betas'
    model_c_basename = 'coeffs'
    model_i_basename = 'intercept'
    tmp_data_basename = 'tmp_data'



    score_method = 'r2'

    # Select hyper parameters for GLM to use for model selection
    # Step 1: Create a dictionary of lists for these relevant keywords...
    kwargs_iterations = {
        'alpha': [0],
        'l1_ratio': [0],

        # 'alpha': [0.0, 0.001, 0.01, 0.1, 1.0],
        # 'l1_ratio': [0.0, 0.0001, 0.001, 0.01],
    }

    # Step 2: Create a dictionary for the fixed keyword arguments that do not require iteration...
    kwargs_fixed = {
        'max_iter': 10000,
        'fit_intercept': False
    }

    # neg_order, pos_order = -14, 14
    # folds = 50
    # folds = 10
    folds = 10
    # folds = 1
    pholdout = 0.2
    pgss = 0.2

    # Step 3: Generate iterable list of keyword sets for possible combinations
    glm_kwarg_lst = sglm_cv.generate_mult_params(kwargs_iterations, kwargs_fixed)


    multi_start = time.time()

    for iXyp, X_y_pairings in enumerate(X_y_pairings_lst):

        widest_orders = [{'X_cols': smf.X_cols_dict_to_default(_['X_cols'], neg_order, pos_order),
                        'y_col': _['y_col']} for _ in X_y_pairings]
        widest_orders = smf.xy_pairs_to_widest_orders(widest_orders)
        max_cols_len = max_cols_len_lst[iXyp]
        plot_rows = plot_rows_lst[iXyp]

        for multifile_fit in multifile_fit_list:
        #         data_folder_join = '_'.join(data_folder.split('/'))

            # prefix = f'{data_folder}-{multifile_fit}-words-recons'

            prefix = f'{data_folder_join}/{multifile_fit}/{base_prefix}_{iXyp}{ft_str}'
            create_all_folders(base_folder, prefix, best_coeffs_folder,
                               best_coeffs_npy_folder, best_reconstruct_folder, best_resids_npy_folder, 
                               best_combined_coeffs_folder, all_models_folder, 
                               all_coeffs_folder, all_reconstruct_folder)



            # Load Signal Data
            signal_files = []
            mouse_names = []
            for wt in wt_used:
                addl_sig_files = glob.glob(pr(f'../../data/interim/{data_folder_src}/GLM_SIGNALS_INTERIM_{wt}_*'))
                signal_files += addl_sig_files
                mouse_names += [wt] * len(addl_sig_files)

            if multifile_fit == 'all':
                combo_dfs, X_cols_sftd, _ = smf.multi_file_analysis_prep(signal_files, widest_orders,)
                combo_fns = ['_'.join(wt_used).replace('WT', '').replace('S', '')]
                mouse_names = combo_fns
            elif multifile_fit == 'by_mouse':
                combo_dfs = []
                X_cols_sftd_lst = []
                combo_fns = []
                mouse_names_2 = []
                for mouse_id in wt_used:
                    mouse_id_files = [_ for _ in signal_files if mouse_id in _]
                    mouse_names_2 += [mouse_id]
                    print('mouse_id', mouse_id)
                    combo_dfs_tmp, X_cols_sftd_tmp, _ = smf.multi_file_analysis_prep(mouse_id_files, widest_orders,)
                    combo_dfs += combo_dfs_tmp
                    X_cols_sftd_lst.append(X_cols_sftd_tmp)
                    combo_fns.append(mouse_id)

                for xcsl in X_cols_sftd_lst:
                    if xcsl != X_cols_sftd_lst[0]:
                        raise ValueError('X_cols_sftd_lst should contain the same elements for every entry')
                mouse_names = mouse_names_2

                X_cols_sftd = X_cols_sftd_lst[0]

            elif multifile_fit == 'single':
                combo_dfs, X_cols_sftd, combo_fns = smf.single_file_analysis_prep(signal_files, widest_orders,)
                mouse_names = mouse_names
            else:
                raise ValueError('multifile_fit must be "all", "single", or "by_mouse"')

            # print(combo_dfs)

            start = time.time()

            results_dict = {}

            for file_num in range(len(combo_dfs)):

                all_coeff_dfs = []
                # Load Table Data
                dfrel_basis = combo_dfs[file_num].reset_index(drop=False).copy()
                fn = combo_fns[file_num].split('.')[0].split('/')[-1]
                mouse_id = mouse_names[file_num]
                dfresids_cols = dfrr_cols



                holdout_score_rnd = None

                full_drop_basis = []
                y_col_lst = []
                for X_y_dct in X_y_pairings:
                    X_cols_sftd_basis = bf.col_shift_bounds_dict_to_col_list(X_y_dct['X_cols'], X_cols_sftd)
                    full_drop_basis += X_cols_sftd_basis
                    if X_y_dct['y_col'][-len('_resid'):] != '_resid':
                        y_col_lst += [X_y_dct['y_col']]
                y_col_drop_basis = sorted(list(set(y_col_lst)))
                full_drop_basis = sorted(list(set(drop_cols_basis + full_drop_basis + y_col_drop_basis)))

                num_cols_na = (dfrel_basis[full_drop_basis].isna().sum(axis=1))
                num_y_0 = (dfrel_basis[y_col_drop_basis] == 0).sum(axis=1)
                has_all_cols = (num_cols_na == 0)&(num_y_0 == 0)

                if has_all_cols.sum() == 0:
                    print(f'No datapoints found for non-NaN dropcols & non-zero ycols for fixed_training: {prefix}_{fn}')
                    continue

                dfrel_basis_has_all_cols = dfrel_basis[has_all_cols]


                for irun in range(num_runs):
                    holdout = models.split_data.holdout_split_by_trial_id(dfrel_basis_has_all_cols,
                                                                          id_cols=['nTrial_filenum'],
                                                                          perc_holdout=pholdout)
                    dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}'] = holdout

                    for iXyd, X_y_dct in enumerate(X_y_pairings):

                        if not fix_training:
                            holdout = models.split_data.holdout_split_by_trial_id(dfrel_basis_has_all_cols, id_cols=['nTrial_filenum'], perc_holdout=pholdout)
                            dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}_iXyd={iXyd}'] = holdout
                        else:
                            dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}_iXyd={iXyd}'] = dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}']



                        X_cols_basis = X_y_dct['X_cols']
                        y_col = X_y_dct['y_col']
                        name = X_y_dct['name']

                        X_cols_sftd_basis = bf.col_shift_bounds_dict_to_col_list(X_cols_basis, X_cols_sftd)
                        run_id = f'{fn}_{y_col}_{iXyd}_run_num={irun}'

                        if (~dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}_iXyd={iXyd}'].isna()).sum() == 0:
                            print(f'No datapoints found for non-NaN dropcols & non-zero ycols for run id: {run_id}.')
                            continue

                        X_cols = X_cols_sftd_basis
                        y_col = y_col

                        df_train = dfrel_basis_has_all_cols[(~holdout)&(dfrel_basis_has_all_cols['wi_trial_keep'])]
                        df_holdout = dfrel_basis_has_all_cols[(holdout)&(dfrel_basis_has_all_cols['wi_trial_keep'])]
                        X_train = df_train[X_cols]
                        y_train = df_train[y_col]
                        X_holdout = df_holdout[X_cols]
                        y_holdout = df_holdout[y_col]

                        kfold_cv_idx = models.split_data.cv_idx_by_trial_id(df_train, trial_id_columns=['nTrial_filenum'], num_folds=folds, test_size=pgss)
                        best_score, best_score_std, best_params, best_model, cv_results = models.sglm_cv.simple_cv_fit(X_train, y_train, kfold_cv_idx, glm_kwarg_lst, model_type='Normal',
                                                                                                                    verbose=0, score_method=score_method)

                        print('Variable Sizes:')
                        for vname, size in sorted(((vname, sys.getsizeof(value)) for vname, value in locals().items()),
                                                 key= lambda x: -x[1])[:30]:
                            print("{:>30}: {:>8}".format(vname, sizeof_fmt(size)))

                        glm, holdout_score, holdout_neg_mse_score = eval.training_fit_holdout_score(X_train, y_train, X_holdout, y_holdout, best_params)

                        betas_df = pd.DataFrame(np.concatenate([np.array([glm.intercept_]), glm.coef_], axis=0).reshape(1,-1),
                                    index=[run_id], columns=['int']+X_cols)
                        betas_df['mouse_id'] = mouse_id
                        betas_df['channel_name'] = y_col + '_' + str(iXyd) + f'_run_num={irun}' if num_runs > 1 else y_col + '_' + str(iXyd)
                        betas_df['name'] = name

                        # Only get R^2 values if only a single model fit
                        assert len(cv_results['full_cv_results']) == 1
                        # TODO: JZ -- IMPLEMENT VERSION FOR MULTIPLE HYPERPARAMETER SWEEPS
                        assert holdout_neg_mse_score == glm.neg_mse_score(X_holdout, y_holdout)

                        betas_df['mse_tr'] = -glm.neg_mse_score(X_train, y_train)
                        betas_df['mse_cv'] = cv_results['full_cv_results'][0]['cv_mse_score']
                        betas_df['mse_te'] = -glm.neg_mse_score(X_holdout, y_holdout) #-holdout_neg_mse_score

                        betas_df['r2_tr'] = glm.r2_score(X_train, y_train)
                        betas_df['r2_cv'] = cv_results['full_cv_results'][0]['cv_R2_score']
                        betas_df['r2_te'] = glm.r2_score(X_holdout, y_holdout) #-holdout_neg_mse_score

                        multi_end = time.time()
                        betas_df['timestamp'] = str(multi_end - multi_start) + ' s'

                        betas_df = betas_df.set_index(['mouse_id', 'channel_name', 'name', 'mse_tr', 'mse_cv', 'mse_te', 'r2_tr', 'r2_cv', 'r2_te', 'timestamp'], append=True)

                        tot_trials_setup = df_train['photometrySideInIndex'].sum()
                        tot_trials_holdout = df_holdout['photometrySideInIndex'].sum()

                        betas_df['AA_cnt_tr'] = df_train['photometrySideInIndexAA'].sum()/tot_trials_setup
                        betas_df['Aa_cnt_tr'] = df_train['photometrySideInIndexAa'].sum()/tot_trials_setup
                        betas_df['aA_cnt_tr'] = df_train['photometrySideInIndexaA'].sum()/tot_trials_setup
                        betas_df['aa_cnt_tr'] = df_train['photometrySideInIndexaa'].sum()/tot_trials_setup
                        betas_df['AB_cnt_tr'] = df_train['photometrySideInIndexAB'].sum()/tot_trials_setup
                        betas_df['Ab_cnt_tr'] = df_train['photometrySideInIndexAb'].sum()/tot_trials_setup
                        betas_df['aB_cnt_tr'] = df_train['photometrySideInIndexaB'].sum()/tot_trials_setup
                        betas_df['ab_cnt_tr'] = df_train['photometrySideInIndexab'].sum()/tot_trials_setup

                        betas_df['AA_cnt_te'] = df_holdout['photometrySideInIndexAA'].sum()/tot_trials_holdout
                        betas_df['Aa_cnt_te'] = df_holdout['photometrySideInIndexAa'].sum()/tot_trials_holdout
                        betas_df['aA_cnt_te'] = df_holdout['photometrySideInIndexaA'].sum()/tot_trials_holdout
                        betas_df['aa_cnt_te'] = df_holdout['photometrySideInIndexaa'].sum()/tot_trials_holdout
                        betas_df['AB_cnt_te'] = df_holdout['photometrySideInIndexAB'].sum()/tot_trials_holdout
                        betas_df['Ab_cnt_te'] = df_holdout['photometrySideInIndexAb'].sum()/tot_trials_holdout
                        betas_df['aB_cnt_te'] = df_holdout['photometrySideInIndexaB'].sum()/tot_trials_holdout
                        betas_df['ab_cnt_te'] = df_holdout['photometrySideInIndexab'].sum()/tot_trials_holdout


                        # betas_df.to_csv(pr(f'{base_folder}/{prefix}/{best_coeffs_npy_folder}/{run_id}_best_coeffs.csv'), index=True, header=True)
                        betas_df.to_hdf(pr(f'{base_folder}/{prefix}/{best_coeffs_npy_folder}/{run_id}_best_coeffs.h5'), key='df', index=True)
                        all_coeff_dfs.append(betas_df)

                        pred_col_name = f'iXyp={iXyp}_irun={irun}_iXyd={iXyd}_name={name}_y={y_col}_irun={irun}' if num_runs > 1 else f'pred_iXyp={iXyp}_irun={irun}_iXyd={iXyd}_name={name}_y={y_col}'
                        dfrel_basis['pred_'+pred_col_name] = pd.Series(glm.predict(dfrel_basis_has_all_cols[X_cols]),
                                                                                     index=dfrel_basis_has_all_cols.index)
                        dfrel_basis['predALL_'+pred_col_name] = glm.predict(dfrel_basis[X_cols])

                        dfresids_cols += [f'holdout_iXyp={iXyp}_irun={irun}_iXyd={iXyd}', 'pred_'+pred_col_name, 'predALL_'+pred_col_name]
#                         break
                    break
    #                 if len(dfrel_basis) != 0 and holdout_score_rnd is not None:
#                         dfrel_basis[dfresids_cols].set_index(['nTrial_filenum'], append=True).to_csv(pr(f'{base_folder}/{prefix}/{best_resids_npy_folder}/{fn}_best_resids_rnum={irun}_iXyp={iXyp}_iXyd={iXyd}_name={name}_all.csv'), index=True, header=True)                
                # dfrel_basis[dfresids_cols].set_index(['nTrial_filenum'], append=True).to_csv(pr(f'{base_folder}/{prefix}/{best_resids_npy_folder}/best_resids_fn={fn}_iXyp={iXyp}_all.csv'), index=True, header=True)
                dfrel_basis[dfresids_cols].set_index(['nTrial_filenum'], append=True).to_hdf(pr(f'{base_folder}/{prefix}/{best_resids_npy_folder}/best_resids_fn={fn}_iXyp={iXyp}_all.h5'), key='df', index=True,)
                combined_best_coeffs = pd.concat(all_coeff_dfs, axis=0)
                combined_best_coeffs.to_hdf(f'{base_folder}/{prefix}/{best_combined_coeffs_folder}/best_coeffs_combo.csv', key='df', index=True)
                break
            break
        break
        break
    return

import cProfile
cprof = cProfile.run('func_for_cprofile()', sort='tottime')

Created directory: C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs\fig1\all\oall-cprofile17-hdf_0-ft
Created directory: C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs\fig1\all\oall-cprofile17-hdf_0-ft\reports\figures\best_coeffs
Created directory: C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs\fig1\all\oall-cprofile17-hdf_0-ft\reports\coeffs\best_coeffs
Created directory: C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs\fig1\all\oall-cprofile17-hdf_0-ft\reports\residuals\best_resids
Created directory: C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs\fig1\all\oall-cprofile17-hdf_0-ft\reports\coeffs\best_combined_coeffs
Created directory: C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs\fig1\all\oall-cprofile17-hdf_0-ft\models\coeffs
Created directory: C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs\fig1\all\oall-cprofile17-hdf_0-ft\models\intercepts
Created directory: C:\Users\Josh\Documents\GitHub\sabati

C:\Users\Josh\anaconda3\envs\sglm\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Josh\anaconda3\envs\sglm\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Josh\anaconda3\envs\sglm\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


{'max_iter': 10000, 'fit_intercept': False, 'alpha': 0, 'l1_ratio': 0}
Running multi
Running multiRunning multi

Running multi
Multirun queue completedMultirun queue completed

Multirun queue completed
{'max_iter': 10000, 'fit_intercept': False, 'alpha': 0, 'l1_ratio': 0}
> cv_mean_score_train: 0.20563975483222308
> cv_R2_score: 0.20445565384806397
> cv_mean_score: 0.20444856687939367
Running multi
Multirun queue completed
Variable Sizes:
                   dfrel_basis: 12.0 GiB
      dfrel_basis_has_all_cols:  8.5 GiB
                      df_train:  5.2 GiB
                       X_train:  1.8 GiB
                    df_holdout:  1.2 GiB
                     X_holdout: 410.2 MiB
                       holdout: 45.8 MiB
                   num_cols_na: 16.9 MiB
                       num_y_0: 16.9 MiB
                       y_train: 14.9 MiB
                     y_holdout:  3.3 MiB
                  has_all_cols:  2.1 MiB
               full_drop_basis:  5.1 KiB
                   X_co

C:\Users\Josh\anaconda3\envs\sglm\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Josh\anaconda3\envs\sglm\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Josh\anaconda3\envs\sglm\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


{'max_iter': 10000, 'fit_intercept': False, 'alpha': 0, 'l1_ratio': 0}
Running multi
Running multi
Running multi
Running multi
Multirun queue completed
Multirun queue completed
Multirun queue completed
{'max_iter': 10000, 'fit_intercept': False, 'alpha': 0, 'l1_ratio': 0}
> cv_mean_score_train: 0.20719794115544313
> cv_R2_score: 0.20636849040901717
> cv_mean_score: 0.20637484960213723
Running multi
Multirun queue completed
Variable Sizes:
                   dfrel_basis: 12.1 GiB
      dfrel_basis_has_all_cols:  8.5 GiB
                      df_train:  5.2 GiB
                       X_train:  1.8 GiB
                    df_holdout:  1.2 GiB
                     X_holdout: 410.2 MiB
                       holdout: 45.8 MiB
                   num_cols_na: 16.9 MiB
                       num_y_0: 16.9 MiB
                       y_train: 14.9 MiB
                     y_holdout:  3.3 MiB
                  has_all_cols:  2.1 MiB
               full_drop_basis:  5.1 KiB
                   X_co

...

        1    0.000    0.000    0.000    0.000 builtins.py:402(OrderedCmpOp)
        1    0.000    0.000    0.000    0.000 npydecl.py:25(Numpy_rules_ufunc)
        2    0.000    0.000    0.000    0.000 target_extension.py:14(__getitem__)
        1    0.000    0.000    0.000    0.000 ltisys.py:300(dlti)
        1    0.000    0.000    0.000    0.000 rcsetup.py:97(<listcomp>)
       10    0.000    0.000    0.000    0.000 rcsetup.py:544(_validate_greaterequal0_lessthan1)
        1    0.000    0.000    0.001    0.001 ElementTree.py:36(_get_py3_cls)
        1    0.000    0.000    0.000    0.000 widgets.py:2490(ToolLineHandles)
        1    0.000    0.000    0.000    0.000 art3d.py:408(Patch3DCollection)
       29    0.000    0.000    0.000    0.000 {method 'groupdict' of 're.Match' objects}
        2    0.000    0.000    0.000    0.000 os.py:688(__delitem__)
       28    0.000    0.000    0.000    0.000 contextlib.py:423(__enter__)
        3    0.000    0.000    0.000    0.000 traitlets.py:92

        1    0.000    0.000    0.000    0.000 __init__.py:1390(_fixupChildren)
        3    0.000    0.000    0.000    0.000 configurable.py:554(initialized)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:600(get_ipython)
        1    0.000    0.000    0.000    0.000 formatters.py:429(for_type)
        2    0.000    0.000    0.000    0.000 stride_tricks.py:25(_maybe_view_as_subclass)
        1    0.000    0.000    0.000    0.000 core.py:185(update_user_library)
        2    0.000    0.000    0.000    0.000 legend_handler.py:337(__init__)
        1    0.000    0.000    0.000    0.000 core.py:817(_DomainTan)
        1    0.000    0.000    0.000    0.000 __init__.py:26(InfinityType)
        7    0.000    0.000    0.000    0.000 expressions.py:64(get_current_context)
        1    0.000    0.000    0.000    0.000 utils.py:210(CacheDict)
        1    0.000    0.000    0.000    0.000 base.py:96(PandasObject)
        1    0.000    0.000    0.000    0.000 apply.py:915(FrameRo

        1    0.000    0.000    0.000    0.000 _cubic.py:464(CubicSpline)
        1    0.000    0.000    0.000    0.000 _discrete_distns.py:574(nhypergeom_gen)
        1    0.000    0.000    0.000    0.000 _multivariate.py:681(multivariate_normal_frozen)
        1    0.000    0.000    0.000    0.000 _multivariate.py:3329(__init__)
        1    0.000    0.000    0.000    0.000 threadpoolctl.py:320(threadpool_limits)
       10    0.000    0.000    0.000    0.000 threadpoolctl.py:381(<genexpr>)
        1    0.000    0.000    0.000    0.000 _bunch.py:1(Bunch)
        1    0.000    0.000    0.000    0.000 _split.py:273(_BaseKFold)
        1    0.000    0.000    0.000    0.000 _data.py:635(StandardScaler)
        1    0.000    0.000    0.000    0.000 _encoders.py:24(_BaseEncoder)
        1    0.000    0.000    0.000    0.000 glm.py:682(TweedieRegressor)
        1    0.000    0.000    0.000    0.000 link.py:166(LogLink)
        1    0.000    0.000    0.000    0.000 _ridge.py:1538(_IdentityClas

        1    0.000    0.000    0.000    0.000 functions.py:632(ExternalFunction)
        1    0.000    0.000    0.000    0.000 iterators.py:5(RangeType)
        1    0.000    0.000    0.000    0.000 npytypes.py:401(NumpyNdIndexType)
        1    0.000    0.000    0.000    0.000 npytypes.py:500(ArrayCTypes)
        1    0.000    0.000    0.000    0.000 npytypes.py:534(ArrayFlags)
        1    0.000    0.000    0.000    0.000 function_type.py:97(UndefinedFunctionType)
        1    0.000    0.000    0.000    0.000 templates.py:957(_IntrinsicTemplate)
        1    0.000    0.000    0.000    0.000 targetconfig.py:160(__init__)
        1    0.000    0.000    0.000    0.000 models.py:157(OmittedArgDataModel)
        1    0.000    0.000    0.000    0.000 models.py:903(OptionalModel)
        3    0.000    0.000    0.000    0.000 imputils.py:108(lower_constant)
        1    0.000    0.000    0.000    0.000 pythonapi.py:76(_ReflectContext)
        1    0.000    0.000    0.000    0.000 generators.

        1    0.000    0.000    0.000    0.000 process.py:144(_ResultItem)
        1    0.000    0.000    0.000    0.000 process.py:573(BrokenProcessPool)
        1    0.000    0.000    0.000    0.000 pool.py:63(ExceptionWithTraceback)
        1    0.000    0.000    0.000    0.000 pool.py:791(MapResult)
        1    0.000    0.000    0.000    0.000 stats.py:3445(F_onewayBadInputSizesWarning)
        1    0.000    0.000    0.000    0.000 stats.py:3871(PearsonRConstantInputWarning)
        1    0.000    0.000    0.000    0.000 stats.py:3881(PearsonRNearConstantInputWarning)
        1    0.000    0.000    0.000    0.000 stats.py:4330(SpearmanRConstantInputWarning)
        1    0.000    0.000    0.000    0.000 decorator.py:254(ContextManager)
        1    0.000    0.000    0.000    0.000 <decorator-gen-2>:1(<module>)
        1    0.000    0.000    0.000    0.000 <decorator-gen-4>:1(<module>)
        1    0.000    0.000    0.000    0.000 misc.py:9(LinAlgWarning)
        1    0.000    0.000  

In [7]:
pr(f'../../data/interim/{data_folder_src}/GLM_SIGNALS_INTERIM_{wt}_*')

NameError: name 'pr' is not defined

In [ ]:
# !conda install numba

In [ ]:
# row_col_keeps['has_all_cols'][0].sum()

dfrel_basis[full_drop_basis]

In [ ]:
# (~dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}_iXyd={iXyd}'].isna()).sum()

In [ ]:
print('Num Training', (~dfrel_basis['holdout_iXyp=0_irun=0'].dropna().astype(bool)).sum())
print('Num Holdouts', (dfrel_basis['holdout_iXyp=0_irun=0'].dropna().astype(bool)).sum())
print('NaN Entries', dfrel_basis['holdout_iXyp=0_irun=0'].isna().sum())

print('Total Entries', (~dfrel_basis['holdout_iXyp=0_irun=0'].dropna().astype(bool)).sum() + (dfrel_basis['holdout_iXyp=0_irun=0'].dropna().astype(bool)).sum() + dfrel_basis['holdout_iXyp=0_irun=0'].isna().sum())
print('Len Entries', len(dfrel_basis))

In [ ]:
len(has_all_cols) - has_all_cols.sum()

In [ ]:
# GLM_SIGNALS_INTERIM_{wt}_dfrel_ft_setup['nTrial'].nunique()

# dfrel_ft_holdout['nTrial'].nunique()

# dfrel_ft_setup['nTrial_filenum'].nunique()

# dfrel_ft_holdout['nTrial_filenum'].nunique()

# dfrel_ft_holdout['nTrial_filenum'].nunique()/dfrel_ft_setup['nTrial_filenum'].nunique()